In [4]:
# !git clone https://github.com/Kolibri25/WINDIFY_AI


fatal: destination path 'WINDIFY_AI' already exists and is not an empty directory.


In [5]:
# cd WINDIFY_AI

/content/WINDIFY_AI/WINDIFY_AI


In [6]:
import my_test as mt

mt.add(2,3)



5